# This Function will reformat and analyse market data from RAVs in order to dimension the Edge Cloud Hub

In [1]:
# Careful this is the third version
def create_clusters(lsee_in, cluster_max,see):
    # Initialize variables and set up first comparison
    lsee = lsee_in.copy()
    # We will use this variable to count towards cluster max size
    count = 0
    # Use low_see to turn off printing in the sub-routines, but leave it on here
    low_see = False
    # Use this variable to record the eNB id of the cluster
    clusterenb = lsee.get_value(0,'Genid')
    start = int(clusterenb)
    if see:
        print 'Starting clustereNb ', start
    if not cluster_max:
        cluster_max = 100
    if see:
        print cluster_max
        # Loop to assign cluster keys every cluster_max groups unless they are the same location
    row_prev_ = get_lon(lsee,0,low_see)
    row_prev = float(row_prev_[1])
    ##################################################

    df_size = len(lsee.index)
    
    for i in range(df_size):
        count +=1
        row_curr_ = get_lon(lsee,i,low_see)
        row_curr = float(row_curr_[1])
        #rowv_,sindex = get_enb(lsee,i,low_see)
        #rowv = int(rowv_[0])
        clusterenb = lsee.get_value(i,'clustereNb')
        ##################################################
        # We could get the distance of the current row to the proposed cluster 
        # If the distance is beyond a limit, we could force a new cluster, this is overkill since
        # others will have a k-means clustering algorithm.
        ##################################################
        # Check 4 cases 
        if row_curr == row_prev and count <cluster_max:
            #add to cluster by taking genid and setting to clusterenb;
            lsee.set_value(i,'clustereNb', start)

            if low_see:
                print "Case 1- Same Site Add to Group \n", 
        elif row_curr == row_prev and count >=cluster_max:
            # add to cluster pointed to by start
            lsee.set_value(i,'clustereNb', start)
            
            if low_see:
                print "Case 2- OverFlow \n"
        elif row_curr != row_prev and count <cluster_max:
            # add to cluster pointed to by start
            lsee.set_value(i,'clustereNb', start)
            
            if low_see:
                print "**************Case 3- New site Add to Group \n", 
        elif row_curr != row_prev and count >=cluster_max:
            #create new cluster; add self; reset count and reset start pointer
            clusterenb = lsee.get_value(i,'Genid')
            start = int(clusterenb)
            if low_see:
                print "start ", start, 'count ', count

            count = 0
            lsee.set_value(i,'clustereNb', start)
            if low_see:
                print "start ", start, 'count ', count
                print "**************Case 4- New Group \n",
            
        else:
            if True:
                print 'Case 5***************'
                print 'This should never be true'
                print 'row_curr ', row_curr, 'row_prev ', row_prev, 'count ', count, 'i ', i
        row_prev = row_curr
    if low_see:
        print 'Test cluster to use ', clusterenb, start, i
    return (lsee)


In [2]:
# simply pythonic way to set the clusereNb id  by eNBid

def get_genid(grp):
    value = grp.get_value(0,0,'Genid')
    grp['clustereNb']= int(value)
    return grp
#lsee is the main dataframe now with the updated cluster eNb ids

In [3]:
def get_df(inputFile):
    import os
    import pandas as pd
    import csv
    # A Place to override the passed filename
    filename = 'RM_MSA_LookUP.csv'
    try:
        with open(inputFile) as mapfile:
            mapfile_df = pd.read_csv(mapfile,skipinitialspace=True,dtype=unicode)
    except:
            print "something went wrong"
    return mapfile_df

In [4]:
def clusterdist(lsee2,see):
    #  2nd Attempt More pythonic way
    see = True
    low_see = False
    import calc_distance
    df_size = len(lsee2.index)
    distance = 0.0


    print "Size of List ", (df_size)

    for row in range(df_size):
        clusterenb = lsee2.get_value(row,'clustereNb')
        lat1 = lsee2.get_value(row,'Lat')
        lon1 = lsee2.get_value(row,'Lon')
        # Need to do a look up to find the clustereNb coordinates
        # Find where pip is in the original dataframe  an index, there may be multiple instance take the first
        index_list = lsee2.loc[lsee2['Genid'] ==clusterenb].index.tolist()
        tcluster = int(index_list[0])

        tlat2 = lsee2.loc[tcluster,['Lat']].values.tolist()
        tlon2 = lsee2.loc[tcluster,['Lon']].values.tolist()
        lat2 = float(tlat2[0])
        lon2 = float(tlon2[0])
        # Call the distance function
        distance = calc_distance.get_dist_to_cell(lat1,lon1,lat2,lon2)
        lsee2.set_value(row,'clusterdist',distance)

        if low_see:
            print 'row', row,'Distance in Km', distance,
            print 'lat1 ','lon1 ', lat1, lon1,  'lat2 ', 'lon2 ', lat2, lon2  
        if distance > 5.0:
            print 'Distance in Km', distance,'clusterenb ', clusterenb," " , row

    return (lsee2)

In [5]:
#Functions to get values from the dataframes based on index
def get_lon (df,test,see):

    # Step 0 of 3
    value_list = df[['Genid','Lon']].values.tolist()
    # Ste 1 of 3 find the index to the first value 2nd column 
    value = value_list[test][1]
    if see:
        print 'get_lon: value ', value
    # still in step 1:
    lon_index = df.loc[df['Lon']==value].index.tolist()
    if see:
        print 'get_lon: lon_index ', lon_index

    # Step 2 of 3
    sindex = lon_index[0]
    if see:
        print 'get_lon: sindex', sindex
    # Step 3A of 3
    row_prev = df.loc[sindex,['Genid','Lon']].values.tolist()
    if see:
        print 'get_lon:  row_prev ', row_prev
    return row_prev
# Repeat to get the eNB id
###################################################
def get_enb (df,test,see):

    # Step 0 of 3
    # Aleady computed above no need to redo
    value_list = df[['Genid','Lon']].values.tolist()
    # Step 1 of 3 find the index to the first value 2nd column 
    value = value_list[test][0]
    if see:
        print 'get_enb: value ', value
    # still in step 1:
    geid_index = df.loc[df['Genid']==value].index.tolist()
    if see:
        print 'get_enb: lon_index ', geid_index

    # Step 2 of 3
    sindex = geid_index[0]
    if see:
        print 'get_enb: sindex', sindex
    # Step 3A of 3
    rowv = df.loc[sindex,['Genid']].values.tolist()
    if see:
        print 'get_enb:  rowv ', rowv
    # let us try modifying a value
    if see:
        print df.get_value(sindex,'clustereNb')
    return rowv, sindex

#Writing to Dataframe

#df.set_value(sindex,'clustereNb', 81414)
#print 'New value ',df.get_value(sindex,'clustereNb')

First we will read in the data using Pandas Data Frames from a raw UTF-8 *.csv file.

In [6]:
import pandas as pd
import numpy as np
import pprint

#Globally enable debug printing
see = False
#%matplotlib notebook
inputFile = "cellsummaryshort.csv"
inputFile = "cellsummaryNYM.csv"
cluster_max = 100

df = get_df(inputFile)

In [7]:
# Read in a list of Columns.
list(df)
pp = pprint.PrettyPrinter(depth=4, width =10)
if see:
    pp.pprint(list(df))

In [8]:
#rename some very long column names
df.rename(columns={'Number of Downlink Antennas':'Tx'}, inplace=True)
df.rename(columns={'Number of Uplink Antennas':'Rx'}, inplace=True)
# Downlink Bandwidth to DLBW
df.rename(columns={ 'Downlink Bandwidth':'DLBW'}, inplace=True)
# Cell Latitude and Cell Longitude to Lat Lon
df.rename(columns={ 'Cell Latitude':'Lat'}, inplace=True)
df.rename(columns={ 'Cell Longitude':'Lon'}, inplace=True)

# get rid of spaces!!
df.rename(columns={ 'Cell Id':'Cellid'}, inplace=True)
df.rename(columns={ 'Global eNodeB ID':'Genid'}, inplace=True)
df.rename(columns={ 'LTE Cell Name':'lteCellid'}, inplace=True)



#clean up Uplink Antennas and Downlink Bandwidth columns to be a number and more readable column
df.replace('ulAntenna1',1, inplace=True)
df.replace('ulAntenna2',2, inplace=True)
df.replace('ulAntenna4',4, inplace=True)
# We could transform these strings to numbers in the future, for now just identifying them is sufficient
df.replace('n75-15MHz', '15MHz', inplace=True)
df.replace('n50-10MHz', '10MHz', inplace=True)
df.replace('n100-20MHz', '20MHz', inplace=True)


# Compute the total number of unique eNBs

count = df['Genid'].groupby(df['Genid']).unique()
count = df['Genid'].unique()
print "Number of eNBs",count.size


#this may not be returning the correct values, the number of unique sites can not be 
# Greater than the total number eNBs!
#countsite = pd.concat([df['Lat'],df['Lon']]).unique()
countsite = np.unique(df[['Lon']])
print "Number of sites", countsite.size
max_enb =count.size

# Compute the total number of unique Cells

enb_df = pd.value_counts(df['Cellid'].values, sort=True)
count = enb_df.values
print ' Number of Cells = ',count.sum()

# Sort the data by lat and longitude.

In [9]:
# Sort Rows by lat, long
# Create a list
cell_sort = []
cell_sort = [ 'Lat', 'Lon']
# Not used
#df_loc_sort = df.sort(cell_sort)
#df.sort(cell_sort, inplace=True)
# Sorting by both Lat and Lon produce interesting results, so we will just choose Lon
try:
    df.sort(['Lon','Genid'], inplace=True)
    df.reset_index(drop = True, inplace=True)
    df= df.convert_objects(convert_numeric=True)
except:
    print 'There was an error sorting in place and converting objects to numerical values'

if see:
    df['Genid'].groupby(df['Cellid']).describe()

Remove columns in the data that are not relevant to the study

In [10]:
# This might change with a different formatted data set if the columns above change.
ndf = df.drop(
 ['\xef\xbb\xbfNetwork Element','eNB ePC Name',
  'eNodeB Name',
 'eNodeB Alias Name',
 'eNodeB Admin State',
 'PLMN Country Code',
 'PLMN Network Code',
 'Physical Cell Identity',
 'LTE Cell Status',
 'Running SW Version',
 'Tracking Area',
 'Cell Radius',
 'Downlink EARFCN',
 'Uplink EARFCN',
  'Uplink Bandwidth',
 'Downlink power',
 '700 MHz',
 '2x2 MIMO',
 '4x2 MIMO',
  'Azimuth'],axis=1)


Add Columns to represent the cluster eNB and cluster id

In [11]:
# Add clustereNB and clusterdist columns
#take the eNb id column and make a list
eNb = []
# One way to populate the clustereNb is to see it with the actual Global eNb Id
eNb = ndf['Genid'].astype(int).tolist()
# now make that list the initial clustereNb column
ndf['clustereNb'] = pd.Series(eNb, index=ndf.index)
# The preferred way is to use random numbers, but we need integers!!
# Override if uncommended with random values.  Either way is fine.
try:
    # now make that list the initial clustereNb column
    ndf['clustereNb'] = pd.Series(eNb, index=ndf.index)
    ndf['clusterdist'] = pd.Series(np.random.randn(), index=ndf.index)

    #ndf['clustereNb'] = pd.Series(np.random.randn(), index=ndf.index)
    #ndf['clustereNb'] = pd.Series(np.random.randint(88888,high=99999), index=ndf.index)
except:
    print 'Random initialization Failed'

if see:
    print ndf[21:28][['Genid','clustereNb','clusterdist']]
    print ndf[0:5][['Genid','clustereNb']]


This provides us with a beginning list of cluster eNB locations equal to the same site. Below we update that list with first eNB in the group by location. Later we will replace the cluster ids with the actual cluster site locations based on N number of clusters combined together.

In [12]:
# temporarliy turning off assigning eNBid by locations
#lsee= ndf.groupby(['Lon']).apply(get_genid)
# it is import to keep the indexing of the DF correct or the clustering routine will fail
lsee =ndf.copy(deep = True)
lsee.reset_index(drop = False, inplace= True)

Next executions set up variables to create a key for the locations and the cluster eNBs
We will loop through the entire set of eNBs grouping by cluster_max to create our sample clusters of cluster_max size. The loop takes care to avoid splitting locations so in some cases the clusters may be slightly bigger than 100 cells.

In [13]:
# Temp overide over cluster max for testing
import time
see = 1; # Turn off printing in the function
start_time = time.time()
# Control whether to perform the long loop for creating clusters
if False:
    lsee2 = create_clusters(lsee,cluster_max,see)
    #Now generate cluster distances
    lsee2 = clusterdist(lsee2,see)
else:
    lsee2 = lsee.copy()

count = lsee2['clustereNb'].unique()
print count.size, 'Number of Unique Clusters of size ~=  ', cluster_max, time.time()- start_time




# Write out to a file using the pandas to_excel function

In [14]:
# First let's create a grouping that provides only the necessary data and unstacks the rows for excel
#This grouping provides a count of cells per cluster with informaton on the antenna config + BW.
grouped =lsee2[['Genid','Cellid','Rx','Tx','clustereNb','DLBW','Lat','Lon','clusterdist']].\
groupby(['clustereNb','Genid','DLBW','Rx','Tx','Lat','Lon','clusterdist']).size().reset_index(name='count')

# Python and pandas requires you perform some operation to turn the groupedby DF into a regular DF. 
# So this is a dummy operation
grouped.sum(inplace = False)

# Build the file name to be used based on the number of clusters chosen
filename = 'NYMCluster-' + str(cluster_max)+ 'cell.xls'
try:
    grouped.to_excel(filename, sheet_name='clusters')
    print 'Wrote ', filename, " to directory in excel format"
except:
    print "File writing didn't succeed"

Wrote  NYMCluster-100cell.xls  to directory in excel format


In [15]:

if see:
    print ' Resulting Before clustering \n',ndf[['Genid','clustereNb','Lat','Lon']][(ndf['Genid']==78778)]
    print ' Resulting after clustering \n',lsee2[['Genid','clustereNb']][(lsee2['clustereNb']==78778)]

 Resulting Before clustering 
      Genid  clustereNb       Lat       Lon
4736  78778       78778  40.01162 -73.76309
 Resulting after clustering 
      Genid  clustereNb
4736  78778       78778


### Use the Madison Square Garden location as a reference point and compute the number of cells within X Km

In [27]:
# coordinates for MSG
# 40.7505° N, 73.9934°
# 40.750298,-73.993324
import calc_distance
df_size = len(lsee2.index)
low_see = False
count = 0
msg_dist = .45
lat_list = []
lon_list = []

for row in range(df_size):

    lat1 = float(40.750298)
    lon1 = float(-73.993324)
    # We will just look for all eNBs in the list 
    tcluster = row

    tlat2 = lsee2.loc[tcluster,['Lat']].values.tolist()
    tlon2 = lsee2.loc[tcluster,['Lon']].values.tolist()
    lat2 = float(tlat2[0])
    lon2 = float(tlon2[0])
    # Call the distance function
    distance = calc_distance.get_dist_to_cell(lat1,lon1,lat2,lon2)
    # Now just make a list of those that within some distance

    if low_see:
        print 'row', row,'Distance in Km', distance,
        print 'lat1 ','lon1 ', lat1, lon1,  'lat2 ', 'lon2 ', lat2, lon2  
        print 'Distance in Km', distance,' Count ', count
    if distance  <= msg_dist:
        count +=1
        lat_list.append(lat2)
        lon_list.append(lon2)


if see:
    print "Final Count = ", count

Final Count =  225


In [31]:
print len(lat_list)
import simplekml
#Add columns
#ndf['clustereNb'] = pd.Series(eNb, index=ndf.index)

#lsee2['clustereNb'].update(pd.Series(temp_list))
#dfloc = pd.Series(np.random.randn(), index=['lat'])
dfloc = pd.DataFrame(lat_list,columns=['lat'])
dfloc['lon'] = pd.Series(lon_list, index=dfloc.index)
countsize = dfloc['lon'].unique()
print countsize.size
#dfloc['lat'].update(pd.Series(lat_list))
print dfloc.shape
 

#Write to a file:
# Build the file name to be used based on the number of clusters chosen
kmlname = 'MadisonSquareGarden-' + str(msg_dist)+ 'cell.kml'
kml = simplekml.Kml()
dfloc.apply(lambda X: kml.newpoint(  coords=[( X["lon"],X["lat"])]) ,axis=1)
try:
    kml.save(path = kmlname)
    print "KML writing to ",kmlname, " succeeded"
except:
    print "KML writing failed"

filename = 'MadisonSquareGarden-' + str(msg_dist)+ 'cell.xls'
try:
    dfloc.to_excel(filename, sheet_name='ToMSG')
    print 'Wrote ', filename, " to directory in excel format"
except:
    print "File writing didn't succeed"


225
39
(225, 2)
KML writing to  MadisonSquareGarden-0.45cell.kml  succeeded
Wrote  MadisonSquareGarden-0.45cell.xls  to directory in excel format


# The remaining are just testing code fragments for display

In [ ]:
#40.54	-74.149
#40.54	-74.1559
distance = calc_distance.get_dist_to_cell(40.54,-74.149,40.54,-74.1559)
if see:
    print 'row', row,'Distance in Km', distance
    print lsee2[46:58][['Genid','clustereNb','clusterdist']]

In [ ]:
# try a new create clusters
ndf.sort('Lon', ascending = False, inplace=True)
df = ndf.copy()
df= df.groupby(['Lon']).apply(get_genid)
df.reset_index(drop = False, inplace=True)

In [ ]:
#
#
#

In [ ]:

if False:
    for name, group in df:
        print 
        for row, col in group.iterrows():
            gentemp = col['clustereNb']
            print gentemp, type(gentemp),'count ', count
            count+=1

        

In [ ]:
# Now write the new clusters into a copy.  Unitl memory and space is an issue, we do this to be safe.
# this is no longer used and could have copied the value incorrectly
#lsee2 = lsee.copy(deep=True)
#lsee2['clustereNb'].update(pd.Series(temp_list))

 

In [ ]:


# better way to look up lat and long
if False:

    lat_list = lsee.loc[teNb,['Genid','Lat']].tolist()
    lon_list = lsee.loc[teNb,['Genid','Lon']].tolist()
    print lat_list, lon_list

    lat_list = lsee.loc[test,['Genid','Lat']].tolist()
    lon_list = lsee.loc[test,['Genid','Lon']].tolist()
    print lat_list, lon_list

#cluster = lsee2['Genid'] == cluster_list[40]
#print lsee2[cluster].get_value(teNb,'Lon')
#lsee2.iloc[['Genid']==79209]['clustereNb']
#distance = calc_distance.get_dist_to_cell(lat1,lon1,lat2,lon2):

In [ ]:
# First Attempt.The calc distance function should only require two sets of lat and longs  2nd Attempt
see = True
low_see = False
import calc_distance
df_size = len(lsee.index)

#find lat and long for a eNb
cluster_list = []
temp_list = []
# Create two list.  We actually want to create clusters counting locations, but use the clustereNB 
# column to keep up with clusters
#Initialize

cluster_list = lsee2[['Genid','clustereNb']].values.tolist()
#Pick a test row
# 31 is good
test = 91
# loop shoot
print len(cluster_list)
for row in range(1,len(cluster_list)):
    test = row
    clusterenb = lsee.get_value(i,'Genid')

    # Get the enB Id from the test row
    pip = cluster_list[test]
    if low_see:
        print 'pip[1] _test_cluster this is an eNbId ', pip[1]
        print 'pip [0] is an eNbId on that same row ', pip[0]

    # Find where pip is in the original dataframe  an index, there may be multiple instance take the first
    index_list = lsee2.loc[lsee2['Genid'] ==pip[1]].index.tolist()
    tcluster = int(index_list[0])
    if low_see:
        print 'Test row for eNB selection ',pip[0],test, 'Cluster row location ', index_list
    # take that reference and get information


    # now get the referenc for eNB
    index_list = lsee2.loc[lsee2['Genid'] ==pip[0]].index.tolist()
    teNb = int(index_list[0])
    if low_see:
        print pip[1], ' teNb ', teNb
        print pip[0], ' tcluster ', tcluster, '\n'
    if low_see:
        print lsee2.loc[teNb,['Genid','Lat','Lon','clustereNb']].tolist()
        print lsee2.loc[tcluster,['Genid','Lat','Lon','clustereNb']].tolist()
   
    tlat1 = lsee2.loc[teNb,['Lat']].values.tolist()
    tlon1 = lsee2.loc[teNb,['Lon']].values.tolist()
    lat1 = float(tlat1[0])
    lon1 = float(tlon1[0])

    tlat2 = lsee2.loc[tcluster,['Lat']].values.tolist()
    tlon2 = lsee2.loc[tcluster,['Lon']].values.tolist()
    lat2 = float(tlat2[0])
    lon2 = float(tlon2[0])
    if low_see:
        print float(tlat1[0])
        print float(tlon1[0])
        print float(tlat2[0])
        print float(tlon2[0])


    distance = calc_distance.get_dist_to_cell(lat1,lon1,lat2,lon2)
    if see:
        print 'row', row,'Distance in Km', distance,
        print 'lat1 ','lon1 ', lat1, lon1,  'lat2 ', 'lon2 ', lat2, lon2

In [ ]:
# Example of using a list to create key columns hard to pass though
key_col = []
key_col = [ 'Global eNodeB ID',
 'Cell Id',
 'Sector',
 'Physical Cell Identity',
 'Cell Latitude',
 'Cell Longitude']
key_col = [ 'Global eNodeB ID']

In [ ]:
lsee2 = lsee.copy()
# Fragment to read a list into a dataframe.

lsee2['clustereNb'].update(pd.Series(temp_list))
# A method to check if the orginal cluster changed.  The new cell may not show up in the resulting clusters.
if False:
    print 'Original \n', ndf[(ndf['Genid']==79209)]
    print ' Resulting after clustering \n',lsee2[(lsee2['Genid']==79209)]
 


grouped =lsee2[['Genid','Cellid','Tx','Rx','clustereNb','DLBW']].groupby(['clustereNb','Tx','Rx','DLBW'], as_index=False)
#lsee2[['Genid','Cellid','Tx','Rx','clustereNb','DLBW']].groupby(['clustereNb','Tx','Rx','DLBW']).count().unstack().unstack()

#lsee2[['Genid','Cellid','clustereNb','DLBW']].groupby(['clustereNb','Genid','DLBW']).count()


table = pd.pivot_table(lsee2, values='Tx',index=['clustereNb'], columns=['DLBW'], aggfunc=np.sum)
table = pd.pivot_table(lsee2, values ='Rx',index=['clustereNb'], columns=['DLBW'])

table

In [ ]:
# This version is defunct and is incorrect
def create_clusters1(lsee, cluster_max):
    
    #print lsee[(lsee['clustereNb']==83209)]
    # Create a Cluster key
    cluster_list = []
    temp_list = []
    # Create two list.  We actually want to create clusters counting locations, but use the clustereNB 
    # column to keep up with clusters
    #Initialize
    
    cluster_list = lsee['clustereNb'].tolist()
    loc_list = lsee['Lon'].tolist()

    count = 0
    if not cluster_max:
        cluster_max = 100
    print cluster_max
    
    row_prev = loc_list[0]
    row_curr = loc_list[0]

    temp_list =  np.copy(cluster_list)

    #print 'First Group @ ', tick, row_prev

    # Loop to assign cluster keys every cluster_max groups unless they are the same location
    for i, row in enumerate(loc_list):
        rowv = cluster_list[i]
        #Should not just write before checking anything!!
        #temp_list[i] = rowv
        count +=1
        # This is the actual value vs index
        row_curr = row
        # We check that the value doesn't equal the previous one because we need to keep eNBs together
        # If we are on a different eNB and we have maxed the cluster then create a new cluster
        if row_curr != row_prev and count > cluster_max:
            #print 'Next Group @ ', i, row, row_curr, row_prev, 'Count ', count
            temp_list[i] = rowv
            count = 0
            row_prev = row_curr

        elif row_curr == row_prev:
            #print 'overflow', row,i, row_curr, row_prev, 'Count ', count
            temp_list[i] = rowv
            count =0
        else:
            temp_list[i] = temp_list[i-1]
    return temp_list

In [ ]:
# Careful this is the second version
def create_clusters2(lsee_in, cluster_max,see):
    # Initialize variables and set up first comparison
    lsee = lsee_in.copy()
    count = 0
    low_see = 0
    start =0
    if not cluster_max:
        cluster_max = 100
    if see:
        print cluster_max
    loc_list = df[['Genid','Lon']].values.tolist()
    # Above row = tupple  = [enid , lon_value]
    loc_list = df[['Lon']].values.tolist()
    # Above row = lon_value

    
    #row_prev = loc_list[0]
    #row_curr = loc_list[0]
    # Use row_prev from new code with test = 0!
    row_prev_ = get_lon(lsee,0,low_see)
    row_prev = row_prev_[1]
    
    # Don't do something like this ever again!!!
    #temp_list =  np.copy(cluster_list)

    # Loop to assign cluster keys every cluster_max groups unless they are the same location
    for i, row in enumerate(loc_list):
        #Should not just write before checking anything!!
        #temp_list[i] = rowv
        rowv,sindex = get_enb(lsee,i,low_see)
        lsee.set_value(sindex,'clustereNb', rowv[0])

        # This is the actual value vs index
        row_curr_ = get_lon(lsee,i,low_see)
        row_curr = float(row_curr_[1])
        # We check that the value doesn't equal the previous one because we need to keep eNBs together
        # If we are on a different eNB and we have maxed the cluster then create a new cluster
        if row_curr != row_prev and count >= cluster_max:
            # in this condition we stay in the same cluster
            if see:
                print 'Next Group @ row ',row,'**i', i,'start ', start, 'row_curr ', \
                row_curr,'row_prev ', row_prev, 'Count ', count
            #temp_list[i] = rowv
            rowv,sindex = get_enb(lsee,i,low_see)
            if see:
                print 'value at count ',lsee.get_value(count,'clustereNb')
                print 'old value ',int(rowv[0])
            # Now right the value of the last group before proceeding
            rowv,sindex = get_enb(lsee,i,low_see)
            value = int(rowv[0])
            lsee.set_value(i,'clustereNb', value)
            if see:
                rowv,sindex = get_enb(lsee,i,low_see)
                print 'New value ',int(rowv[0])
            count = 0
            start = i
            if see:
                print 'Start ', start
            row_curr_ = get_lon(lsee,i,low_see)
            row_prev = float(row_curr_[1])
        if row_curr == row_prev and count < cluster_max:
            if see:
                print 'create_clusters: overflow  row', row,'**start ', start,'i' ,\
                i,'row_curr ', row_curr, row_prev, 'Count ', count
            #temp_list[i] = rowv
            rowv,sindex = get_enb(lsee,i,low_see)
            if see:
                print 'old value ',rowv[0]
            
            rowv,sindex = get_enb(lsee,start,low_see)
            value = int(rowv[0])
            lsee.set_value(i,'clustereNb', value)
            if see:
                print 'New value ',int(rowv[0])
            count +=1
        if row_curr != row_prev:
            if see:
                print 'Continue in group: row', row,'**start ',start, 'i ',i,'row_curr ', \
                row_curr,'row_prev ',row_prev,'Count ', count
            rowv,sindex = get_enb(lsee,i,low_see)
            if see:
                print 'old value ',int(rowv[0])
            rowv,sindex = get_enb(lsee,start,low_see)
            
            value = int(rowv[0])
            lsee.set_value(i,'clustereNb', value)
            
            if see:
                print 'New value ',int(rowv[0])
            row_prev = row_curr
            count +=1
        else:
            if see:
                print 'Count ', count, ' We missed you i', i,'row_curr', row_curr,\
                'row_prev',row_prev, 'start ', start
    if see:
        count = lsee['clustereNb'].unique()
    print count.size, 'Number of Unique Clusters of size ~=  ', cluster_max
    return (lsee)
